In [62]:
import os
import tarfile
import urllib
import pandas as pd
import numpy as np
import sklearn
import datetime

<h1>Read pre-processed smart meter database</h1>

In [2]:
file_path = os.path.dirname(os.path.abspath(""))

In [3]:
preprocessed_dataset_folder = os.path.abspath(os.path.join(file_path, "./data/preprocessed"))
preprossed_data_pickle_path = preprocessed_dataset_folder+ "./preprocessed_smdb_pickle.pkl"

In [139]:
# Read pickle version of pre-processed smart meter dataset
smart_meter_df = pd.read_pickle(preprossed_data_pickle_path)

In [140]:
LCLid_list = smart_meter_df["LCLid"].unique().tolist()
# Reduce sample size for faster analysis
smart_meter_df = smart_meter_df[smart_meter_df["LCLid"].isin(LCLid_list[:int(len(LCLid_list)/1000)])].reset_index().drop(columns = ["index"])

In [141]:
smart_meter_df

,LCLid,DateTime,Consumption,Acorn,Acorn_grouped,Month,Day,Weekday,Hour,precipitation,temperature,irradiance_surface,irradiance_toa,snowfall,snow_mass,cloud_cover,air_density
0,MAC000002,2013-01-01 00:00:00,0.219,ACORN-A,Affluent,1,1,1,0,0.0806,3.886,0.0,0.0,0.0047,0.0544,0.4855,1.2269
1,MAC000002,2013-01-01 01:00:00,0.191,ACORN-A,Affluent,1,1,1,1,0.0787,3.695,0.0,0.0,0.0066,0.0603,0.4675,1.2284
2,MAC000002,2013-01-01 02:00:00,0.182,ACORN-A,Affluent,1,1,1,2,0.0857,3.539,0.0,0.0,0.0050,0.0659,0.4681,1.2300
3,MAC000002,2013-01-01 03:00:00,0.194,ACORN-A,Affluent,1,1,1,3,0.0864,3.419,0.0,0.0,0.0039,0.0699,0.4824,1.2313
4,MAC000002,2013-01-01 04:00:00,0.122,ACORN-A,Affluent,1,1,1,4,0.0842,3.336,0.0,0.0,0.0045,0.0739,0.4986,1.2324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8767,MAC000002,2013-12-31 19:00:00,0.252,ACORN-A,Affluent,12,31,1,19,0.0901,4.564,0.0,0.0,0.0000,0.0981,0.6670,1.2237
8768,MAC000002,2013-12-31 20:00:00,0.833,ACORN-A,Affluent,12,31,1,20,0.0892,4.537,0.0,0.0,0.0000,0.0947,0.6786,1.2239
8769,MAC000002,2013-12-31 21:00:00,0.332,ACORN-A,Affluent,12,31,1,21,0.0805,4.547,0.0,0.0,0.0000,0.0912,0.6814,1.2238
8770,MAC000002,2013-12-31 22:00:00,0.456,ACORN-A,Affluent,12,31,1,22,0.0858,4.581,0.0,0.0,0.0000,0.0878,0.6884,1.2235


<h1>Train-test-split</h1>

In [142]:
from sklearn.model_selection import StratifiedShuffleSplit

In [143]:
# Split trin and test set with stratified sampling based on Acorn_grouped to ensure the same distribution of acron groups
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(smart_meter_df, smart_meter_df["Acorn_grouped"]):
    strat_train_set = smart_meter_df.loc[train_index]
    strat_test_set = smart_meter_df.loc[test_index]

In [144]:
smart_meter_train = strat_train_set.copy()
smart_meter_test = strat_test_set.copy()

In [145]:
smart_meter_train

,LCLid,DateTime,Consumption,Acorn,Acorn_grouped,Month,Day,Weekday,Hour,precipitation,temperature,irradiance_surface,irradiance_toa,snowfall,snow_mass,cloud_cover,air_density
6442,MAC000002,2013-09-26 01:00:00,0.196,ACORN-A,Affluent,9,26,3,1,0.0259,9.682,0.0000,0.0000,0.0000,0.0000,0.7825,1.2210
8536,MAC000002,2013-12-22 05:00:00,0.095,ACORN-A,Affluent,12,22,6,5,0.3859,4.816,0.0000,0.0000,0.0201,0.0669,0.7527,1.2274
4919,MAC000002,2013-07-24 17:00:00,0.115,ACORN-A,Affluent,7,24,2,17,0.0464,20.438,258.8353,492.9004,0.0000,0.0000,0.6412,1.1757
1488,MAC000002,2013-03-03 22:00:00,0.930,ACORN-A,Affluent,3,3,6,22,0.0071,1.953,0.0000,0.0000,0.0002,0.0156,0.7476,1.2617
697,MAC000002,2013-01-30 00:00:00,0.202,ACORN-A,Affluent,1,30,2,0,0.4990,7.375,0.0000,0.0000,0.0000,0.5687,0.8131,1.2049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7508,MAC000002,2013-11-09 10:00:00,0.176,ACORN-A,Affluent,11,9,5,10,0.2236,5.171,114.2102,375.1923,0.0015,0.0374,0.7009,1.2332
5610,MAC000002,2013-08-22 11:00:00,0.391,ACORN-A,Affluent,8,22,3,11,0.0966,19.400,597.1196,968.9520,0.0000,0.0000,0.2661,1.1925
5291,MAC000002,2013-08-09 04:00:00,0.083,ACORN-A,Affluent,8,9,4,4,0.1419,13.269,2.4995,14.2563,0.0000,0.0000,0.4718,1.2057
4419,MAC000002,2013-07-03 21:00:00,0.161,ACORN-A,Affluent,7,3,2,21,0.0443,12.548,0.0174,0.2467,0.0000,0.0000,0.3058,1.2042


<h1>Data exploration</h1>

In [146]:
smart_meter_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7017 entries, 6442 to 2623
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   LCLid               7017 non-null   object        
 1   DateTime            7017 non-null   datetime64[ns]
 2   Consumption         7017 non-null   float64       
 3   Acorn               7017 non-null   object        
 4   Acorn_grouped       7017 non-null   object        
 5   Month               7017 non-null   int64         
 6   Day                 7017 non-null   int64         
 7   Weekday             7017 non-null   int64         
 8   Hour                7017 non-null   int64         
 9   precipitation       7017 non-null   float64       
 10  temperature         7017 non-null   float64       
 11  irradiance_surface  7017 non-null   float64       
 12  irradiance_toa      7017 non-null   float64       
 13  snowfall            7017 non-null   float64  

In [147]:
smart_meter_train.describe()

,Consumption,Month,Day,Weekday,Hour,precipitation,temperature,irradiance_surface,irradiance_toa,snowfall,snow_mass,cloud_cover,air_density
count,7017.000000,7017.000000,7017.000000,7017.000000,7017.000000,7017.000000,7017.000000,7017.000000,7017.000000,7017.000000,7017.000000,7017.000000,7017.000000
mean,0.246666,6.517458,15.717686,3.009976,11.546245,0.103164,8.415761,131.179143,264.338626,0.006214,0.543350,0.629646,1.229174
std,0.252105,3.453802,8.812776,2.002966,6.945393,0.144959,5.589410,197.085718,347.011054,0.033218,1.705595,0.216214,0.026570
min,0.065000,1.000000,1.000000,0.000000,0.000000,0.000000,-2.482000,0.000000,0.000000,0.000000,0.000000,0.035700,1.157200
25%,0.111000,4.000000,8.000000,1.000000,6.000000,0.015300,3.929000,0.000000,0.000000,0.000000,0.000000,0.468000,1.208900
50%,0.152000,7.000000,16.000000,3.000000,12.000000,0.047700,8.090000,5.704200,28.332900,0.000000,0.000000,0.637000,1.226100
75%,0.264000,10.000000,23.000000,5.000000,18.000000,0.132800,12.697000,212.049900,489.127200,0.000500,0.160700,0.807400,1.251000
max,2.994000,12.000000,31.000000,6.000000,23.000000,1.893200,23.502000,849.489700,1128.031800,0.915300,16.114300,0.996100,1.294300


<h1>More pre-processing</h1>

In [148]:
# For some columns, e.g. Month, hour, weekday, create subgroups to contain each unique value as too much categories take too much time
# This will later be incorporated into python model feature.py to replace month, day and hour with season, dau_tupe and time_slot

In [149]:
smart_meter_train = smart_meter_train.drop(columns = ["Month", "Day", "Weekday", "Hour"])

<h3>Create season feature</h3>

In [150]:
def create_season_feature(df):
    # Create a series date contining a numerical representation of the day of DateTime (e.g. the start date of spring, 21 of March = 321)
    date = df.DateTime.dt.month*100 + df.DateTime.dt.day

    # Assign season feature by putting date value into bins
    # Since pd.cut only takes unique bin labels and winter months is across the end and the beginning of the year, added a space  after "Winter" and strip() afterwards

    df['Season'] = pd.cut(date,[0,321,620,922,1220,1300],
                       labels=["Winter","Spring","Summer","Autumn","Winter "]).str.strip()
    return df

In [151]:
smart_meter_train = create_season_feature(smart_meter_train)

In [152]:
def create_day_type_feature(df):
    # Group days of week into (later to improve to include specific bank holidays from the UK 2013 calendar)
    
    # Assign day_type feature by putting day of week value into bins

    df["Day_type"] = pd.cut(df["DateTime"].dt.weekday,[-1,4,5, 6],
                       labels=["Weekday","Day before holiday","Holiday"])
    
    holiday_list = [datetime.datetime(2013, 1, 1), 
                   datetime.datetime(2013, 3, 29), 
                   datetime.datetime(2013, 4, 1), 
                   datetime.datetime(2013, 5, 6), 
                   datetime.datetime(2013, 8, 26), 
                   datetime.datetime(2013, 12, 25), 
                   datetime.datetime(2013, 12, 26)]
    
    day_before_holiday_list = [datetime.datetime(2013, 3, 28),
                              datetime.datetime(2013, 12, 24)]
    
    # Specific bank holidays from the UK 2013 calendar
    df.loc[df["DateTime"].isin(holiday_list), "Day_type"] = "Holiday"
    df.loc[df["DateTime"].isin(day_before_holiday_list), "Day_type"] = "Day before holiday"
    

    return df

In [153]:
smart_meter_train = create_day_type_feature(smart_meter_train)

In [154]:
#smart_meter_train =smart_meter_train.drop(columns = ["Time_slot_x", "Time_slot_y"])

In [155]:
def create_time_slot_feature(df):
    # Assign season feature by putting hours in a day into time slot bins (Midnight, early morning, morning, early afternoon, late afternoon, early evening, late evening)
    # Since pd.cut only takes unique bin labels and winter months is across the end and the beginning of the year, added a space  after "Winter" and strip() afterwards

    df["Time_slot"] = pd.cut(df["DateTime"].dt.hour,[-1,3, 6, 11, 14, 17, 20, 23],
                       labels=["Midnight", 
                               "Early morning", 
                               "Morning", 
                               "Early afternoon", 
                               "Late afternoon", 
                               "Early evening", 
                               "Late evening"])
    return df

In [156]:
smart_meter_train = create_time_slot_feature(smart_meter_train)

In [157]:
smart_meter_train

,LCLid,DateTime,Consumption,Acorn,Acorn_grouped,precipitation,temperature,irradiance_surface,irradiance_toa,snowfall,snow_mass,cloud_cover,air_density,Season,Day_type,Time_slot
6442,MAC000002,2013-09-26 01:00:00,0.196,ACORN-A,Affluent,0.0259,9.682,0.0000,0.0000,0.0000,0.0000,0.7825,1.2210,Autumn,Weekday,Midnight
8536,MAC000002,2013-12-22 05:00:00,0.095,ACORN-A,Affluent,0.3859,4.816,0.0000,0.0000,0.0201,0.0669,0.7527,1.2274,Winter,Holiday,Early morning
4919,MAC000002,2013-07-24 17:00:00,0.115,ACORN-A,Affluent,0.0464,20.438,258.8353,492.9004,0.0000,0.0000,0.6412,1.1757,Summer,Weekday,Late afternoon
1488,MAC000002,2013-03-03 22:00:00,0.930,ACORN-A,Affluent,0.0071,1.953,0.0000,0.0000,0.0002,0.0156,0.7476,1.2617,Winter,Holiday,Late evening
697,MAC000002,2013-01-30 00:00:00,0.202,ACORN-A,Affluent,0.4990,7.375,0.0000,0.0000,0.0000,0.5687,0.8131,1.2049,Winter,Weekday,Midnight
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7508,MAC000002,2013-11-09 10:00:00,0.176,ACORN-A,Affluent,0.2236,5.171,114.2102,375.1923,0.0015,0.0374,0.7009,1.2332,Autumn,Day before holiday,Morning
5610,MAC000002,2013-08-22 11:00:00,0.391,ACORN-A,Affluent,0.0966,19.400,597.1196,968.9520,0.0000,0.0000,0.2661,1.1925,Summer,Weekday,Morning
5291,MAC000002,2013-08-09 04:00:00,0.083,ACORN-A,Affluent,0.1419,13.269,2.4995,14.2563,0.0000,0.0000,0.4718,1.2057,Summer,Weekday,Early morning
4419,MAC000002,2013-07-03 21:00:00,0.161,ACORN-A,Affluent,0.0443,12.548,0.0174,0.2467,0.0000,0.0000,0.3058,1.2042,Summer,Weekday,Late evening


<h1>Transformation Pipelines</h1>

In [158]:
# Remove unnecessary columns
smart_meter = smart_meter_train.drop(columns = ["Consumption", "LCLid", "DateTime", "Acorn"])

<h3>Categorical data encoding</h3>

In [159]:
from sklearn.preprocessing import OneHotEncoder

In [161]:
cat_attribs = ["Season", "Day_type", "Time_slot","Acorn_grouped"]

# Extract categorical columns from smart meter database
smart_meter_cat = smart_meter[cat_attribs]

In [162]:
# Perform categorical encoding for the categorical data
cat_encoder = OneHotEncoder()
smart_meter_cat_onehot = cat_encoder.fit_transform(smart_meter_cat)

<h3>Numerical data missing data imputation</h3>

In [165]:
from sklearn.impute import SimpleImputer

In [171]:
# Extract numerical columns from smart meter database
smart_meter_num = smart_meter.drop(columns = cat_attribs)

In [172]:
imputer = SimpleImputer(strategy = "median")
smart_meter_num_impute = imputer.fit_transform(smart_meter_num)

<h3>Transformation pipeline for numerical data</h3>

In [173]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [174]:
# Create transformation pipeline for numerical columns
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy = "median")),  
    ("std_scaler", StandardScaler()),
    ])

In [175]:
# create num_attribs as a list of numerical attributes for transformation
num_attribs = list(smart_meter_num)

In [176]:
from sklearn.compose import ColumnTransformer

In [177]:
# Create full pipeline of data transformation
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs), 
    ("cat", OneHotEncoder(), cat_attribs), 
    ])

In [180]:
# Apply transformation pipeline to dataset
smart_meter_prepared = full_pipeline.fit_transform(smart_meter)

<h1>Model selection</h1>

In [181]:
smart_meter_labels = smart_meter_train["Consumption"].copy()

In [182]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [183]:
forest_reg = RandomForestRegressor()

In [184]:
forest_reg.fit(smart_meter_prepared,smart_meter_labels)

RandomForestRegressor()

In [185]:
smart_meter_predictions = forest_reg.predict(smart_meter_prepared)

In [186]:
forest_mse = mean_squared_error(smart_meter_labels, smart_meter_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

0.0797372018314608

In [187]:
# get importance
importance = forest_reg.feature_importances_

In [188]:
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.12228
Feature: 1, Score: 0.12675
Feature: 2, Score: 0.05148
Feature: 3, Score: 0.06461
Feature: 4, Score: 0.03642
Feature: 5, Score: 0.10369
Feature: 6, Score: 0.12559
Feature: 7, Score: 0.15084
Feature: 8, Score: 0.00916
Feature: 9, Score: 0.01032
Feature: 10, Score: 0.00624
Feature: 11, Score: 0.00875
Feature: 12, Score: 0.00699
Feature: 13, Score: 0.00714
Feature: 14, Score: 0.00861
Feature: 15, Score: 0.00383
Feature: 16, Score: 0.09977
Feature: 17, Score: 0.02043
Feature: 18, Score: 0.00266
Feature: 19, Score: 0.02698
Feature: 20, Score: 0.00112
Feature: 21, Score: 0.00633
Feature: 22, Score: 0.00000
